# Build End-to-End RAG System using Fine-tuned Models

Here we will build a simple RAG pipeline which will use:

- Custom Fine-tuned Embedder
- Custom Fine-tuned Llama 3 LLM



## Load RAG Dataset

In [1]:
from datasets import load_dataset

dataset = load_dataset("dipanjanS/india_rag_wikidata", split="train")

In [2]:
df = dataset.to_pandas()
df.head(10)

,article_id,context,question,answer
0,73985,"Basil (""Ocimum basilicum"") ( or ) is a plant o...",What is Basil commonly known as?,"Dear Friend, Basil is commonly known as Sweet ..."
1,73985,"Basil (""Ocimum basilicum"") ( or ) is a plant o...",Where is Basil originally native to?,"Dear Friend, Basil is originally native to Ind..."
2,259745,The Roerich Pact is a treaty on Protection of ...,What is the significance of the Roerich Pact?,"Dear Friend, the Roerich Pact is significant b..."
3,259745,The Roerich Pact is a treaty on Protection of ...,When was the Roerich Pact signed?,"Dear Friend, the Roerich Pact was signed on 15..."
4,207506,The Indian Air Force is the air arm of the Ind...,What is the history of the Indian Air Force?,"Dear Friend, the Indian Air Force was founded ..."
5,207506,The Indian Air Force is the air arm of the Ind...,What role does the Indian Air Force play?,"Dear Friend, the Indian Air Force serves as th..."
6,4954,Prem Pal Singh Rawat (in India called Maharaji...,Who is Prem Pal Singh Rawat?,"Dear Friend, Prem Pal Singh Rawat, also known ..."
7,4954,Prem Pal Singh Rawat (in India called Maharaji...,What organizations has Prem Pal Singh Rawat be...,"Dear Friend, Prem Pal Singh Rawat has been ass..."
8,5113,Chennai (formerly known as Madras) is the capi...,What is significant about Chennai's history?,"Dear Friend, Chennai, formerly known as Madras..."
9,5113,Chennai (formerly known as Madras) is the capi...,Where is Chennai located?,"Dear Friend, Chennai is located on the Coroman..."


## Get Context Document Corpus

In [3]:
context_corpus = df['context'].drop_duplicates().tolist()
len(context_corpus)

669

In [4]:
context_corpus[:2]

['Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silky leaves 3–5\xa0cm long and 1–3\xa0cm broad. The leaves are opposite each other. The flowers are quite big. They are white in color and arranged as a spike.',
 'The Roerich Pact is a treaty on Protection of Artistic and Scientific Institutions and Historic Monuments, signed by the representatives of 21 states in the Oval Office of the White House on 15 April 1935. As of January 1, 1990, the Roerich Pact had been ratified by ten nations: Brazil, Chile, Colombia, Cuba, the 

## Load up Custom Fine-tuned BGE Embedder Model

Here we load the embedder model we fine-tuned earlier

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# model_name = "dipanjanS/bge-base-en-v1.5-fte"
model_name = "./bge-base-en-v1.5-fte"
ft_embedder = HuggingFaceEmbeddings(model_name=model_name)

## Create Vector Database Index

In [6]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
db = Chroma.from_texts(texts=context_corpus,
                       collection_name='wikipedia_db',
                       embedding=ft_embedder,
                       collection_metadata={"hnsw:space": "cosine"})

In [7]:
retriever = db.as_retriever(search_type="similarity",
                            search_kwargs={"k": 3})

## Test Sample Retrievals

In [8]:
query = "Tell me about the Indian flag"

docs = retriever.invoke(query)
docs

[Document(page_content='The modern Flag of The Republic of India has three colours, which are placed horizontally. At the top is saffron, which signifies sacrifice and patriotism. In the middle is white, which stands for truth in word and actions and purity in our thoughts. At the bottom is green, which stands for life and prosperity. In the middle of the white is a blue wheel, which is called the Ashoka Chakra. It has 24 spokes and it stands for progress.The Chakra or the wheel also symbolizes the Power of the State governed by Dharma. It is also called the tiranga or tricolour. The flag was discovered by Venkayya Pingali.'),
 Document(page_content='The modern Republic of India (Hindi:); has several official National symbols including a historical document, a flag, an emblem, an anthem, a memorial tower as well as several national heroes. All the symbols were picked up at various times. The design of the national flag was officially adopted by the Constituent Assembly just 21 days bef

## Create a Retrieval Function on the Vector DB

In [9]:
def rag_retrieval(query):
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

query = "Tell me about the national anthem of India"
print(rag_retrieval(query))

The modern Republic of India (Hindi:); has several official National symbols including a historical document, a flag, an emblem, an anthem, a memorial tower as well as several national heroes. All the symbols were picked up at various times. The design of the national flag was officially adopted by the Constituent Assembly just 21 days before Independence, on the 24th of July in 1947. There are also several other patriotic symbols including the national animal, bird, fruit, flower and tree... have all been selected carefully to project the image of India at its best. They are chosen to reflect Indian culture and beliefs and also the positive at - tributes often associated with Indian traditions respectively.

"Jana Gana Mana" (जन-गण-मन) is the title for the national anthem of India. In 1911, it was composed and originally written in Bengali by popular Indic poet and artist Rabindranath Tagore—the same author for the Bangladeshi anthem ""Amar Shonar Bangla"". Originally a five-verse Bra

## Load Fine-tuned Llama 3 LLM

Here we will use our custom fine-tuned Llama 3 8B LLM from the previous hands-on demo

In [10]:
from unsloth import FastLanguageModel

model_name = "RAG_Llama3-8B-it"
# model_name = "dipanjanS/RAG_Llama3-8B-it"
max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = None,
        load_in_4bit = False,
    )
FastLanguageModel.for_inference(model)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Create a simple RAG prompt

In [11]:
def rag_qa_prompt(question, context):
    """
    Creates a chat prompt for the Llama model.

    Args:
        question: The question to be answered.
        context: The context information to be used for generating the answer.

    Returns:
        A string containing the chat template.
    """

    prompt = f"""\
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    Use the context information below only to answer the given question.
    Do not make up answers.

    Context: {context}

    Question: {question}

    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """
    # Remove any leading whitespace characters from each line in the prompt.
    prompt = "\n".join([line.lstrip() for line in prompt.splitlines()])
    return prompt

## Create a LLM Response Generation Function

In [12]:
def rag_generation(model, tokenizer, prompt, max_tokens=512):
    """
    Generates a response from the Llama model using the given prompt.

    Args:
        prompt: The prompt to be used for generating the response.

    Returns:
        A string containing the generated response.
    """

    # Encode the prompt.
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    # Generate the output.
    output = model.generate(**inputs, max_new_tokens=max_tokens,
                            eos_token_id=tokenizer.eos_token_id,
                            tokenizer=tokenizer,
                            stop_strings=["<|eot_id|>"])
    # Decode the output.
    text = tokenizer.decode(output[0], skip_special_tokens=False)
    return text

## Create the RAG Pipeline

Here we will put everything together and chain the three functions one after the other:

- RAG retrieval from Vector DB
- Pass context docs with user question to RAG prompt
- Send prompt to LLM and generate response

In [13]:
def rag_pipeline(query, model, tokenizer, max_tokens=512):
    context = rag_retrieval(query)
    prompt = rag_qa_prompt(query, context)
    response = rag_generation(model, tokenizer,
                              prompt, max_tokens)
    response = (response.split('<|start_header_id|>assistant<|end_header_id|>')[1]
                       .split('<|im_end|>')[0])
    return response

## Test out the RAG system

In [14]:
query = "Tell me about the Indian flag"
result = rag_pipeline(query, model, tokenizer)
print(result)


Answer: Dear Friend, the modern Flag of India is a tricolor consisting of three colors placed horizontally. The top section is saffron, representing sacrifice and patriotism. The middle section is white, symbolizing truth and purity, while the bottom section is green, representing life and prosperity. In the middle of the white section is a blue Ashoka Chakra, which has 24 spokes and signifies progress. 


In [15]:
query = "What is the capital of India?"
result = rag_pipeline(query, model, tokenizer)
print(result)


Answer: Dear Friend, the capital of India is New Delhi. It is a union territory within the megacity of Delhi and has a rich history. 


In [16]:
query = "What is the capital of India? Tell me in detail"
result = rag_pipeline(query, model, tokenizer)
print(result)


Answer: Dear Friend, the capital of India is New Delhi, which is a union territory. It is part of the megacity of Delhi and has a rich history that makes it a significant cultural hub. 


In [17]:
query = "Tell me about the national anthem of India"
result = rag_pipeline(query, model, tokenizer)
print(result)


Answer: Dear Friend, the national anthem of India is called 'Jana Gana Mana', which was composed by Rabindranath Tagore in 1911. It was originally a five-verse Brahmo hymn in Bengali called 'Bharoto Bhagyo Bidhata'. 


In [18]:
query = "Tell me about the national song of India"
result = rag_pipeline(query, model, tokenizer)
print(result)


Answer: Dear Friend, the national song of India is called 'Vande Mataram'. It was written by Bankim Chandra Chattopadhyay in 1876 and published in 1882. The title translates to 'Long live the Motherland'. 


#### Frameworks to evaluate RAG systems

- Trulens
- LLM as a Judge
- Ragas
- Deepeval